In [2]:
import numpy as np
import plotly.graph_objects as go
from scipy.signal import find_peaks

# utils and helper_files
from utils.get_raw_data import get_multiple_data_arrays
from helper_files.normalizations import normalize_high_peak
from helper_files.gaussian_fitting import gaussian
from helper_files.peak_ratios import split_arrs, channel_to_kev, find_peaks_arrs, remove_linear_bg, n_gaussians_and_m_order_bg, print_table, make_fit, find_nearest, fwhm, table_values_FWHM, plotly_table


In [3]:
ga = get_multiple_data_arrays(filters=['GaAs'], normalize=False)
# highest peak normalization
norm = normalize_high_peak(ga)
norm, names = split_arrs(norm)
s = norm[3]

In [4]:
dispersion = 0.0100283 # from GaAs, Ga La and As Ka peaks
offset = 21.079
deg = 12 # degree of the polynomial to the background
prominence = 0.01 # prominence of the peaks, for the find_peaks
pixel_removal = 50 # removal of pixels before and after each peak
channel_points = 2048
Vacc = 30

channels = np.arange(0, channel_points)
res_increase = 10 # resolution increase for fit
channels_high_res = np.linspace(0, len(channels), len(channels) * res_increase)
# to center the high_res on channels, we need to shift it by half res_increase
# channels_high_res -= res_increase / 20

keV = channel_to_kev(arr=channels, dispersion=dispersion, offset=offset)
keV_high_res = channel_to_kev(arr=channels_high_res, dispersion=dispersion, offset=offset)

title = 'GaAs'

# the keV of the peaks for the table
gaLa = 1.098
asLa = 1.282
gaKa = 9.251
asKa = 10.543
gaKb = 10.264
asKb = 11.726

# p_names = ["Ga La", "As La", "Ga Ka", "As Ka", "Ga Kb", "As Kb"]
poi = [gaLa, asLa, gaKa, asKa, gaKb, asKb] # poi, peak of interest

In [5]:
peaks, _ = find_peaks(s, prominence=prominence)
# fig = go.Figure()
# fig.update_layout(title="Plot on channels, find the calibration peaks")
# fig.add_trace(go.Scatter(x=channels, y=s, mode="markers", name=names[0]))
# for p in peaks:
#     fig.add_vline(x=p, line_width=1, line_dash="dash", line_color="red", annotation_text=f"{p}")
# fig

In [6]:
bg40 = remove_linear_bg(s, peaks, pixel_removal=40, offset=int(offset))
bg100 = remove_linear_bg(s, peaks, pixel_removal=100, offset=int(offset))

fig = go.Figure()
fig.update_layout(title="Plot on channels, find the calibration peaks")
# fig.add_trace(go.Scatter(x=channels, y=s, mode="lines", name='raw'))
fig.add_trace(go.Scatter(x=keV, y=bg40, mode="lines", name='bg'))
fig.add_trace(go.Scatter(x=keV, y=bg100, mode="lines", name='bg'))
for p in peaks:
    pk = channel_to_kev(p, dispersion=dispersion, offset=offset)
    # fig.add_vline(x=p, line_width=1, line_dash="dash", line_color="red", annotation_text=f"{p}")
    fig.add_vline(x=pk, line_width=1, line_dash="dash", line_color="grey")

fig.update_layout(title=title, xaxis_title='keV', yaxis_title='Normalized intensity')

'2'


'2'

In [7]:
def make_fit_on_fig(fig, order=12, prominence=0.01):
    fit_vals, _ = make_fit(s, channels, deg=order, prominence=prominence)
    bg_fit = np.polyval(fit_vals[:order+1], channels)
        # fig.add_trace(go.Scatter(x=keV, y=bg_fit, mode="lines", name=f'order {order} bg'))
    fit = n_gaussians_and_m_order_bg(channels, order, *fit_vals)
    # fig.add_trace(go.Scatter(x=keV, y=fit, mode="lines", name=f'order {order} fit'))
    print(f'rms error for order {order}: {np.sqrt(np.mean((s - fit)**2)):.5f}')
    return fit, bg_fit


def add_fit(fig, fit, bg_fit, order):
    fig.add_trace(go.Scatter(x=keV, y=bg_fit, mode="lines", name=f'order {order} bg'))
    fig.add_trace(go.Scatter(x=keV, y=fit, mode="lines", name=f'order {order} fit'))

def fit_fig(f6, bg6, f12, bg12):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=keV, y=s, mode="markers", name='Raw'))
    add_fit(fig, f6, bg6, 6)
    add_fit(fig, f12, bg12, 12)
    fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))
    fig.update_layout(width=800, height=400)
    fig.update_layout(legend=dict(x=0.75, y=0.90))
    fig.update_layout(font=dict(size=18))
    fig.update_layout(showlegend=True)

    return fig

def rmse_range(fit, start, stop):
    return np.sqrt(np.mean((s[start:stop] - fit[start:stop])**2))


f6, bg6 = make_fit_on_fig(fig, order=6)
f12, bg12 = make_fit_on_fig(fig, order=12)

rms error for order 6: 0.00157
rms error for order 12: 0.00140


In [13]:
print(f12)

[-0.00420987 -0.00380955 -0.00341754 ...  0.00114818  0.0011756
  0.00120428]


In [8]:
fig = fit_fig(f6, bg6, f12, bg12)
start = 21
stop = 2048
rmse6 = rmse_range(f6, start, stop)
rmse12 = rmse_range(f12, start, stop)

start_keV = (start-offset)*dispersion
stop_keV = (stop-offset)*dispersion

# fig.update_layout(showlegend=False)
fig.add_annotation(x=0.4, y=0.95, xref="paper", yref="paper", text=f"RMSE for {np.abs(start_keV):.2f}-{stop_keV:.2f} keV", showarrow=False, font=dict(size=18))
fig.add_annotation(x=0.4, y=0.90, xref="paper", yref="paper", text=f"{'order 6:':<11} {rmse6:.5f}", showarrow=False, font=dict(size=18))
fig.add_annotation(x=0.4, y=0.85, xref="paper", yref="paper", text=f"{'order 12:':<10} {rmse12:.5f}", showarrow=False, font=dict(size=18))

# fig.add_annotation(x=0.4, y=0.95, xref="paper", yref="paper", text=f"RMSE order {'6':3}: {rmse6:.5f}", showarrow=False, font=dict(size=18))
# fig.add_annotation(x=0.4, y=0.90, xref="paper", yref="paper", text=f"RMSE order 12: {rmse12:.5f}", showarrow=False, font=dict(size=18))
fig.update_layout(xaxis_title='Energy [keV]', yaxis_title='Relative intensity [a.u.]')
fig.update_yaxes(range=[-0.02, 1.02])


'h'
# fig.write_image("../plots/fit/fit_GaAs30kV_whole.png", scale=2)

'h'

In [9]:
start = 910
stop = 1220
fig = fit_fig(f6, bg6, f12, bg12)
rmse6 = rmse_range(f6, start, stop)
rmse12 = rmse_range(f12, start, stop)
start_keV = (start-offset)*dispersion
stop_keV = (stop-offset)*dispersion

fig.update_layout(showlegend=False)
fig.add_annotation(x=0.96, y=0.95, xref="paper", yref="paper", text=f"RMSE for {start_keV:.2f}-{stop_keV:.2f} keV", showarrow=False, font=dict(size=18))
fig.add_annotation(x=0.96, y=0.90, xref="paper", yref="paper", text=f"{'order 6:':<11} {rmse6:.5f}", showarrow=False, font=dict(size=18))
fig.add_annotation(x=0.96, y=0.85, xref="paper", yref="paper", text=f"{'order 12:':<10} {rmse12:.5f}", showarrow=False, font=dict(size=18))
# fig.add_annotation(x=0.85, y=0.95, xref="paper", yref="paper", text=f"RMSE order {'6':3}: {rmse6:.5f}", showarrow=False, font=dict(size=18))
# fig.add_annotation(x=0.85, y=0.90, xref="paper", yref="paper", text=f"RMSE order 12: {rmse12:.5f}", showarrow=False, font=dict(size=18))
fig.update_xaxes(range=[8.9, 12.0])
fig.update_yaxes(range=[-0.0005, 0.38])

fig.write_image("../plots/fit/fit_GaAs30kV_K.png", scale=2)


In [10]:
fig = fit_fig(f6, bg6, f12, bg12)

start = 30
stop = 215
fig = fit_fig(f6, bg6, f12, bg12)
rmse6 = rmse_range(f6, start, stop)
rmse12 = rmse_range(f12, start, stop)

# fig.update_layout(showlegend=False)
# fig.add_annotation(x=0.15, y=0.95, xref="paper", yref="paper", text=f"RMSE order {'6':3}: {rmse6:.5f}", showarrow=False, font=dict(size=18))
# fig.add_annotation(x=0.15, y=0.90, xref="paper", yref="paper", text=f"RMSE order 12: {rmse12:.5f}", showarrow=False, font=dict(size=18))

start_keV = (start-offset)*dispersion
stop_keV = (stop-offset)*dispersion

fig.update_layout(showlegend=False)
fig.add_annotation(x=0.10, y=0.95, xref="paper", yref="paper", text=f"RMSE for {start_keV:.2f}-{stop_keV:.2f} keV", showarrow=False, font=dict(size=18))
fig.add_annotation(x=0.10, y=0.90, xref="paper", yref="paper", text=f"{'order 6:':<11} {rmse6:.5f}", showarrow=False, font=dict(size=18))
fig.add_annotation(x=0.10, y=0.85, xref="paper", yref="paper", text=f"{'order 12:':<10} {rmse12:.5f}", showarrow=False, font=dict(size=18))
fig.update_xaxes(range=[-0.1, 1.9])
fig.update_yaxes(range=[-0.0005, 0.5])

fig.write_image("../plots/fit/fit_GaAs30kV_L.png", scale=2)


In [11]:
fig = fit_fig(f6, bg6, f12, bg12)
fig.update_xaxes(range=[-0.1, 10])
fig.update_yaxes(range=[-0.0005, 0.025])

fig = fit_fig(f6, bg6, f12, bg12)

start = 22
stop = 270
fig = fit_fig(f6, bg6, f12, bg12)
rmse6 = rmse_range(f6, start, stop)
rmse12 = rmse_range(f12, start, stop)

start_keV = (start-offset)*dispersion
stop_keV = (stop-offset)*dispersion

fig.update_layout(showlegend=False)
fig.add_annotation(x=0.96, y=0.95, xref="paper", yref="paper", text=f"RMSE for {start_keV:.2f}-{stop_keV:.2f} keV", showarrow=False, font=dict(size=18))
fig.add_annotation(x=0.96, y=0.90, xref="paper", yref="paper", text=f"{'order 6:':<11} {rmse6:.5f}", showarrow=False, font=dict(size=18))
fig.add_annotation(x=0.96, y=0.85, xref="paper", yref="paper", text=f"{'order 12:':<10} {rmse12:.5f}", showarrow=False, font=dict(size=18))

fig.update_xaxes(range=[-0.1, 3.7])
fig.update_yaxes(range=[-0.0005, 0.025])

fig.write_image("../plots/fit/fit_GaAs30kV_bg.png", scale=2)
